In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import time
import re

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [2]:
# URL del artículo en Wayback Machine
url = "https://web.archive.org/web/20180304015632/https://www.elespectador.com/opinion/independencia-en-ceros-columna-742192"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Diccionario para meses en español
meses = {
    "Ene": "enero", "Feb": "febrero", "Mar": "marzo", "Abr": "abril",
    "May": "mayo", "Jun": "junio", "Jul": "julio", "Ago": "agosto",
    "Sep": "septiembre", "Oct": "octubre", "Nov": "noviembre", "Dic": "diciembre"
}

# 1. Extraer título
titulo = soup.find('h1').get_text(strip=True)

# 2. Extraer fecha formateada
fecha_element = soup.find('div', class_='node-post-date')
if fecha_element:
    fecha_texto = fecha_element.get_text(strip=True).split(' - ')[0]
    dia, mes_abrev, anio = fecha_texto.split()
    fecha_formateada = f"{dia} de {meses[mes_abrev]} de {anio}"
else:
    fecha_formateada = "Fecha no encontrada"

# 3. Extraer autor (texto después de "Por:")
autor_element = soup.find('span', class_='by')  # Localiza el span con "Por:"
if autor_element:
    autor = autor_element.next_sibling.strip()  # Toma el texto HERMANO siguiente al span
else:
    autor = "Autor no encontrado"

# 4. Extraer contenido limpio con párrafos (versión mejorada)
contenido_div = soup.find('div', class_='node-body')
if contenido_div:
    # Primero eliminar el div no deseado si existe
    info_node = contenido_div.find('div', class_='info_node_hide')
    if info_node:
        info_node.decompose()  # Esto elimina completamente el div y su contenido

    # Eliminar solo elementos no deseados (scripts, iframes, etc.)
    for element in contenido_div(['script', 'style', 'iframe', 'img', 'figure']):
        element.decompose()

    # Procesar cada párrafo conservando formato semántico
    parrafos = []
    for p in contenido_div.find_all('p'):
        # Extraer todo el texto del párrafo incluyendo etiquetas de formato
        texto_parrafo = p.get_text(' ', strip=True)  # El espacio une elementos separados
        if texto_parrafo:
            # Limpieza final de espacios múltiples
            texto_parrafo = ' '.join(texto_parrafo.split())
            parrafos.append(texto_parrafo)

    contenido = '\n\n'.join(parrafos)
else:
    contenido = "Contenido no encontrado"

# Resultados
print(f"Título: {titulo}")
print(f"Fecha: {fecha_formateada}")
print(f"Autor: {autor}")
print("\n--- CONTENIDO ---\n")
print(contenido)

Título: Independencia en ceros
Fecha: 2 de marzo de 2018
Autor: José Roberto Acosta

--- CONTENIDO ---

Hace una semana para nadie era un problema los tres ceros en nuestros billetes, pero para ocultar su inoperancia y conflicto de intereses en el caso Odebrecht, el fiscal general Néstor Humberto Martinez salió con tan costosa cortina de humo.

Es sospechoso que, siendo legal y necesaria la independencia del fiscal general de la Nación respecto al Poder Ejecutivo del Estado, de manera simultánea saliera el ministro de Hacienda a decir que ya tenía listo el proyecto de ley parta tramitarlo en el Congreso generándole un gasto de por lo menos $400.000 millones a la nación, sin contar los costos en cambios y software y sistemas de contabilidad para las empresas y las millonadas que derrochará el Gobierno en campañas publicitarias de pedagogía por los tres años que duraría la transición al “nuevo peso”, tiempo suficiente para la operación de lavado de las caletas mencionadas por el fiscal.


In [12]:
# ESTE es el que estoy usando para procesar múltiples URLs normales

# Usamos with para que el archivo se cierre automáticamente
with open("urls.txt", "r") as f:
    # Leemos todas las líneas y filtramos las vacías
    urls = [line.strip() for line in f if line.strip()]

# Eliminamos duplicados
urls = list(dict.fromkeys(urls))

# Diccionario para meses en español
meses = {
    "Ene": "enero", "Feb": "febrero", "Mar": "marzo", "Abr": "abril",
    "May": "mayo", "Jun": "junio", "Jul": "julio", "Ago": "agosto",
    "Sep": "septiembre", "Oct": "octubre", "Nov": "noviembre", "Dic": "diciembre"
}

# Lista para almacenar todos los resultados
datos = []
n = 0
for url in urls:
    try:
        print(f"Procesando: {url}")
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 1. Extraer título
        titulo = soup.find('h1').get_text(strip=True) if soup.find('h1') else "Título no encontrado"

        # 2. Extraer fecha formateada
        fecha_element = soup.find('div', class_='node-post-date')
        if fecha_element:
            fecha_texto = fecha_element.get_text(strip=True).split(' - ')[0]
            try:
                dia, mes_abrev, anio = fecha_texto.split()
                fecha_formateada = f"{dia} de {meses[mes_abrev]} de {anio}"
            except:
                fecha_formateada = fecha_texto
        else:
            fecha_formateada = "Fecha no encontrada"

        # 3. Extraer autor
        autor_element = soup.find('span', class_='by')
        if autor_element:
            autor = autor_element.next_sibling.strip() if autor_element.next_sibling else "Autor no encontrado"
        else:
            autor = "Autor no encontrado"

        # 4. Extraer contenido limpio con párrafos (versión mejorada)
        contenido_div = soup.find('div', class_='node-body')
        if contenido_div:
            # Primero eliminar el div no deseado si existe
            info_node = contenido_div.find('div', class_='info_node_hide')
            if info_node:
                info_node.decompose()  # Esto elimina completamente el div y su contenido

            # Eliminar solo elementos no deseados (scripts, iframes, etc.)
            for element in contenido_div(['script', 'style', 'iframe', 'img', 'figure']):
                element.decompose()

            # Procesar cada párrafo conservando formato semántico
            parrafos = []
            for p in contenido_div.find_all('p'):
                # Extraer todo el texto del párrafo incluyendo etiquetas de formato
                texto_parrafo = p.get_text(' ', strip=True)  # El espacio une elementos separados
                if texto_parrafo:
                    # Limpieza final de espacios múltiples
                    texto_parrafo = ' '.join(texto_parrafo.split())
                    parrafos.append(texto_parrafo)

            contenido = '\n\n'.join(parrafos)
        else:
            contenido = "Contenido no encontrado"

        # Agregar a la lista de datos
        datos.append({
            'Autor': autor,
            'Fecha': fecha_formateada,
            'Título': titulo,
            'Contenido': contenido,
            'URL': url
        })
        #time.sleep(1)
    except Exception as e:
        print(f"Error procesando {url}: {str(e)}")
        datos.append({
            'Autor': f"Error: {str(e)}",
            'Fecha': "",
            'Título': "",
            'Contenido': "",
            'URL': url
        })
    n = n + 1
    if n % 20 == 0 and n < len(urls):
        print(f"Procesados: {n} de {len(urls)}")
        time.sleep(150)



# Crear DataFrame y guardar como CSV
df = pd.DataFrame(datos)

# Ordenar columnas
column_order = ['Autor', 'Fecha', 'Título', 'Contenido', 'URL']
df = df[column_order]

# Crear archivo Excel
nombre_archivo = "resultados_articulos1.xlsx"
with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Artículos')

    # Ajustar el ancho de las columnas
    worksheet = writer.sheets['Artículos']
    worksheet.column_dimensions['A'].width = 25  # Autor
    worksheet.column_dimensions['B'].width = 20  # Fecha
    worksheet.column_dimensions['C'].width = 40  # Título
    worksheet.column_dimensions['D'].width = 80  # Contenido
    worksheet.column_dimensions['E'].width = 60  # URL

print(f"\nProceso completado. Resultados guardados en {nombre_archivo}")

Procesando: https://web.archive.org/web/20200719121010/https://www.elespectador.com/opinion/un-acuerdo-nacional-y-racional/
Procesando: https://web.archive.org/web/20200719121010/https://www.elespectador.com/opinion/estados-unidos-ahogo-en-alcohol-el-futuro-de-sus-ninos/
Procesando: https://web.archive.org/web/20200719121010/https://www.elespectador.com/opinion/no-haga-olas-alcaldesa-lopez/
Procesando: https://web.archive.org/web/20200719121010/https://www.elespectador.com/opinion/el-nuevo-contexto-de-la-equidad/
Procesando: https://web.archive.org/web/20200719121010/https://www.elespectador.com/opinion/el-retorno-de-los-autos-de-fe/
Procesando: https://web.archive.org/web/20200719121010/https://www.elespectador.com/opinion/una-ley-estatutaria-para-enfrentar-la-pandemia/
Procesando: https://web.archive.org/web/20200719121010/https://www.elespectador.com/opinion/las-jefas-del-hogar/
Procesando: https://web.archive.org/web/20200719121010/https://www.elespectador.com/opinion/el-arbol-invi

KeyboardInterrupt: 

# Hecho por Andrés


In [4]:
# el que hizo Andrés 
import requests
import re
import json
import time
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
 
# Mapeo de meses en inglés a español (para el formato de fecha)
meses_es = {
    'January': 'enero', 'February': 'febrero', 'March': 'marzo', 
    'April': 'abril', 'May': 'mayo', 'June': 'junio', 
    'July': 'julio', 'August': 'agosto', 'September': 'septiembre', 
    'October': 'octubre', 'November': 'noviembre', 'December': 'diciembre'
}

with open('urls.txt', 'r', encoding='utf-8') as f:
    urls = [line.strip() for line in f if line.strip()]
 
articulos = []
errores = []


def formatear_fecha(fecha_str):
    try:
        # Parsear la fecha original (asumiendo formato ISO o similar)
        fecha = datetime.strptime(fecha_str, "%Y-%m-%dT%H:%M:%SZ")
        # Formatear como "día de mes de año" en español
        dia = fecha.day
        mes = meses_es[fecha.strftime('%B')]
        año = fecha.year
        return f"{dia} de {mes} de {año}"
    except:
        # Si falla el formateo, devolver la fecha original
        return fecha_str
    

def procesar_url(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            raise Exception(f"HTTP {response.status_code}")
 
        html = response.text
        match = re.search(r'Fusion\.globalContent\s*=\s*({.*?})\s*;\s*Fusion\.globalContentConfig', html, re.DOTALL)
        if not match:
            raise Exception("No se encontró Fusion.globalContent")
 
        raw_json = match.group(1)
        data = json.loads(raw_json)
 
        titulo = data.get("headlines", {}).get("basic", "")
        fecha = formatear_fecha(data.get("display_date", ""))
        autor = data.get("credits", {}).get("by", [{}])[0].get("name", "")
 
        elementos = data.get("content_elements", [])
        parrafos = []
        for e in elementos:
            if e.get("type") == "text":
                html_content = e.get("content", "")
                texto = BeautifulSoup(html_content, "html.parser").get_text()
                parrafos.append(texto.strip())
 
        texto_completo = "\n\n".join(parrafos)
 
        return {
            "Autor": autor,
            "Fecha": fecha,
            "Título": titulo,
            "Texto": texto_completo,
            "URL": url
        }
 
    except Exception as e:
        errores.append((url, str(e)))
        return None
   
for i, url in enumerate(urls):
    print(f"Procesando {i+1}/{len(urls)}: {url}")
    resultado = procesar_url(url)
    if resultado:
        articulos.append(resultado)
       
    time.sleep(3)
 
    if (i + 1) % 20 == 0:
        print("Pausa de 2 minutos para evitar bloqueo...")
        time.sleep(120)
 
df = pd.DataFrame(articulos)
df.to_excel("resultados_articulos1.xlsx", index=False)
 
with open("errores_scraping.txt", 'w', encoding='utf-8') as f:
    for url, error in errores:
        f.write(f"{url} \n")
 
print("Proceso finalizado. Artículos guardados.")

Procesando 1/1026: https://web.archive.org/web/20201101134752mp_/https://www.elespectador.com/opinion/amenaza-fascista/
Procesando 2/1026: https://web.archive.org/web/20201101134752mp_/https://www.elespectador.com/opinion/el-covid-19-y-las-enfermedades-mentales/
Procesando 3/1026: https://web.archive.org/web/20201101134752mp_/https://www.elespectador.com/opinion/zarpazo-judicial/
Procesando 4/1026: https://web.archive.org/web/20201101134752mp_/https://www.elespectador.com/opinion/elogio-de-un-conservador/
Procesando 5/1026: https://web.archive.org/web/20201101134752mp_/https://www.elespectador.com/opinion/la-opinion-de-los-colombianos/
Procesando 6/1026: https://web.archive.org/web/20201101134752mp_/https://www.elespectador.com/opinion/deshacer-el-nudo-ii/
Procesando 7/1026: https://web.archive.org/web/20201101134752mp_/https://www.elespectador.com/opinion/naturaleza-juridica-del-acuerdo/
Procesando 8/1026: https://web.archive.org/web/20201101134752mp_/https://www.elespectador.com/opin

# Para extraer links

In [9]:
import requests
import re
import json
from urllib.parse import urljoin
from time import sleep

def get_opinion_links(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    opinion_links = set()
    
    try:
        print(f"Procesando: {url}")
        response = requests.get(url, headers=headers, timeout=15)
        if response.status_code != 200:
            print(f"Error HTTP {response.status_code}")
            return []
        
        html = response.text
        
        # Método 1: Buscar en el JSON embebido (como en tu ejemplo exitoso)
        json_data = extract_from_embedded_json(html)
        if json_data:
            links_from_json = find_links_in_json(json_data)
            opinion_links.update(links_from_json)
        
        # Método 2: Búsqueda directa en HTML como respaldo
        links_from_html = find_links_in_html(html, url)
        opinion_links.update(links_from_html)
        
        return sorted(opinion_links)
    
    except Exception as e:
        print(f"Error procesando {url}: {str(e)}")
        return []

def extract_from_embedded_json(html):
    """Extrae el JSON embebido como en tu ejemplo exitoso"""
    try:
        match = re.search(r'Fusion\.globalContent\s*=\s*({.*?})\s*;\s*Fusion\.globalContentConfig', html, re.DOTALL)
        if not match:
            return None
        
        raw_json = match.group(1)
        return json.loads(raw_json)
    except:
        return None

def find_links_in_json(json_data):
    """Busca enlaces en la estructura JSON"""
    links = set()
    
    # Buscar en content_elements
    elements = json_data.get("content_elements", [])
    for element in elements:
        if element.get("type") == "link":
            href = element.get("url", "")
            if href and "opinion" in href.lower():
                links.add(href)
        
        # Buscar en el contenido HTML dentro de elementos
        content = element.get("content", "")
        if content:
            found = re.findall(r'href=["\'](https?://[^"\']*opinion[^"\']*)["\']', content, re.IGNORECASE)
            links.update(found)
    
    return links

def find_links_in_html(html, base_url):
    """Búsqueda directa en el HTML como respaldo"""
    links = set()
    
    # Buscar todos los enlaces que contengan "opinion"
    found = re.findall(r'href=["\']([^"\']*opinion[^"\']*)["\']', html, re.IGNORECASE)
    
    for link in found:
        # Convertir a URL absoluta si es relativa
        absolute_link = urljoin(base_url, link)
        links.add(absolute_link)
    
    return links

# Ejemplo de uso
wayback_url = "https://web.archive.org/web/20200707160217/https://www.elespectador.com/opinion/"
opinion_links = get_opinion_links(wayback_url)

print(f"\nEncontrados {len(opinion_links)} enlaces con 'opinion':")
for i, link in enumerate(opinion_links, 1):
    print(f"{i}. {link}")

# Guardar resultados
if opinion_links:
    with open('enlaces_opinion.txt', 'w', encoding='utf-8') as f:
        f.write("\n".join(opinion_links))
    print("\nResultados guardados en 'enlaces_opinion.txt'")
else:
    print("\nNo se encontraron enlaces. Recomendaciones:")
    print("1. Verifica que la URL de Wayback Machine tenga el contenido esperado")
    print("2. Prueba con otra fecha de archivo")
    print("3. Inspecciona manualmente la página para ver la estructura de los enlaces")

Procesando: https://web.archive.org/web/20200707160217/https://www.elespectador.com/opinion/

Encontrados 8 enlaces con 'opinion':
1. https://web.archive.org/web/20190707024117/https://www.elespectador.com/opinion
2. https://web.archive.org/web/20200607142257/https://www.elespectador.com/opinion/
3. https://web.archive.org/web/20200706153249/https://www.elespectador.com/opinion/
4. https://web.archive.org/web/20200707160217/http://web.archive.org/screenshot/https://www.elespectador.com/opinion/
5. https://web.archive.org/web/20200707160217/https://www.elespectador.com/opinion/
6. https://web.archive.org/web/20200708161029/https://www.elespectador.com/opinion
7. https://web.archive.org/web/20200808002313/https://www.elespectador.com/opinion
8. https://web.archive.org/web/20210707185227/https://www.elespectador.com/opinion/

Resultados guardados en 'enlaces_opinion.txt'
